In [1]:
import glob
import fiftyone as fo
import os
import requests
import json
from pathlib import Path
from eyepop import EyePopSdk
from eyepop.data.data_endpoint import DataEndpoint
from eyepop.data.data_jobs import DataJob
from eyepop.data.data_types import DatasetCreate, AssetImport, AutoAnnotateParams, Dataset, Asset, ChangeEvent, ChangeType, DatasetUpdate, UserReview, Model, ModelCreate, ModelStatus
import asyncio
from tqdm.notebook import tqdm
import hashlib
from fiftyone import Dataset
from fiftyone.core.labels import Detection, Detections
from fiftyone.core.metadata import ImageMetadata

# 🔧 Configuration
accountUUID = "034cb8e37f5444e98a78f1be65fd0bff"
fullModelUUID = "068819c3a65473ce800081b210833ff7"
fullModelQuantizedUUID = "" #technically same 
# https://localhost:3000/wizardModel?type=object&step=deploy&accountUUID=49326f2e085a46c39ba73f91c52e436c&modelUUID=067ad32ae1bf704b80003bc57fbe3694&datasetUUID=066f5aed921579b280009378da7c0049
sub300ModelUUID = "068818bdef1c7a3f800056a0dafbe761"
sub300ModelQuantizedUUID = "" # haven't quantized yet

datasetUUID = "068818885c547b1e80002649e95254ce"
fullDatasetQuantizedUUID = ""
sub300datasetUUID = "0688195453827ae280000526e30dc7a9"
sub300datasetQuantizedUUID = "" # haven't quantized yet

modelLabel = {
    datasetUUID: "EyePop.ai Trained Model (full)",
    fullDatasetQuantizedUUID: "EyePop.ai Trained Model (full, quantized)",
    sub300datasetUUID: "EyePop.ai Trained Model (sub-300)",
    sub300datasetQuantizedUUID: "EyePop.ai Trained Model (sub-300, quantized)",
}

#apikey = "AAE_w6lCcrCa27chNAbZO-WdZ0FBQUFBQmwyUFk5bmtLZnJBQ2RFVWVDbzU1MnkwTUMzYXhQWjA4a0ZEczFKWWdONjdRS0NGWUZ5aF90aXVQZ3FrcWdkZWwwUEx6Q0luM0F3b3ItMjdqRmhUQkxyTWVvSndFLWRCUENjZGNlanZhbGhRTDdtV289"
apikey = "AAGcsWj8N2PlKQl9c9ydz3QFZ0FBQUFBQm1mZDB5eDUwalNlYi12NWotd3hsVGJiMW1sVXF1dE9aOU9oSGVBOWtBQXoxZmNjUE5Nb1YzY3RROUdzbVUwUkZtcDhZcG5vSWROTzR1TU8ybGhZckx6RTgzYVZwMjZEREZjalZubnpYaUNMWVdBODg9"
cache_directory = "./.cache/voxel51/" + accountUUID + "/" + sub300datasetUUID

def checkCacheDirectory():
    if not os.path.exists(os.path.expanduser(cache_directory)):
        print("Cache directory does not exist. Creating it...")
        os.makedirs(os.path.expanduser(cache_directory))
    cache_files = os.listdir(os.path.expanduser(cache_directory))
    if not cache_files:
        print("Cache directory is empty.")
        return False
    print("Cache directory contains files:", cache_files)    
    return True

def convert_annotations_to_coco(asset_uuid, annotations, image_id, starting_annotation_id=1):
    if(not annotations or len(annotations) == 0):
        print("No annotations found.")
        return None
    
    image_width = annotations[0].annotation.source_height
    image_height = annotations[0].annotation.source_height
    
    coco = {
        "images": [
            {
                "id": image_id,
                "width": image_width,
                "height": image_height,
                "file_name": f"{asset_uuid}.jpg",
            }
        ],
        "annotations": [],
        "categories": []
    }

    category_name_to_id = {}
    annotation_id = starting_annotation_id

    for annotation in annotations:
        if not hasattr(annotation, "annotation") or not hasattr(annotation.annotation, "objects"):
            continue

        for obj in annotation.annotation.objects:
            label = obj.classLabel
            if label not in category_name_to_id:
                category_id = len(category_name_to_id) + 1
                category_name_to_id[label] = category_id
                coco["categories"].append({
                    "id": category_id,
                    "name": label,
                    "supercategory": "none"
                })
            else:
                category_id = category_name_to_id[label]

            bbox = [
                obj.x,
                obj.y,
                obj.width,
                obj.height
            ]
            area = obj.width * obj.height

            coco["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox": bbox,
                "area": area,
                "iscrowd": 0
            })
            annotation_id += 1

    return coco

async def downloadDatasetToCache(accountUUID, datasetUUID):
    print("Downloading dataset from EyePop.ai to cache directory...")
    print("Account UUID:", accountUUID)
    print("Dataset UUID:", datasetUUID)
    cache_directory = "./.cache/voxel51/" + accountUUID + "/" + datasetUUID
    print("Cache directory:", cache_directory)
    print("Using API key:", apikey)



    async with EyePopSdk.dataEndpoint(
        #eyepop_url = "https://dataset-api.staging.eyepop.xyz/",
        eyepop_url = "https://web-api.staging.eyepop.xyz/",
        secret_key=apikey, 
        account_id=accountUUID, 
        is_async=True, 
        disable_ws=False) as endpoint:

        asset_list = await endpoint.list_assets(dataset_uuid=datasetUUID, include_annotations=True)
        print(f"Found {len(asset_list)} assets in the dataset.")
        
        os.makedirs(os.path.expanduser(cache_directory), exist_ok=True)
        cache_path = Path(os.path.expanduser(cache_directory))
        images_dir = cache_path / "data"
        annotations_dir = cache_path / "annotations"
        images_dir.mkdir(exist_ok=True)
        annotations_dir.mkdir(exist_ok=True)

        # write entire asset_list to a JSON file for debugging
        asset_list_path = cache_path / "eyepop_asset_list.json"
        asset_list_val_path = cache_path / "eyepop_asset_list_val.json"
        def default_serializer(obj):
            if hasattr(obj, "isoformat"):
                return obj.isoformat()
            raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")
        with open(asset_list_path, "w") as f:
            json.dump([asset.model_dump() for asset in asset_list], f, indent=2, default=default_serializer)
        print(f"Asset list saved to {asset_list_path}")

        # make a copy of the asset_list and filter out assets that do not have "partition": "val"
        asset_list_val = [asset for asset in asset_list if asset.partition == "val"]
        with open(asset_list_val_path, "w") as f:
            json.dump([asset.model_dump() for asset in asset_list_val], f, indent=2, default=default_serializer)
        print(f"Asset list VAL saved to {asset_list_val_path}")

        combined_coco = {
            "images": [],
            "annotations": [],
            "categories": []
        }
        category_name_to_id = {}
        uuid_to_image_id = {}
        annotation_id = 1
        next_image_id = 1

        pbar = tqdm(asset_list, desc="Starting downloads")
        for asset in pbar:
            pbar.set_description(f"Downloading {asset.uuid}")

            image_path = images_dir / f"{asset.uuid}.jpg"
            if not image_path.exists():
                image_response = await endpoint.download_asset(asset.uuid, datasetUUID, dataset_version=None)
                image_bytes = await image_response.read()
                with open(image_path, "wb") as f:
                    f.write(image_bytes)

            # Assign numeric image_id
            image_id = next_image_id
            uuid_to_image_id[asset.uuid] = image_id
            next_image_id += 1

            metadata = convert_annotations_to_coco(
                asset_uuid=asset.uuid,
                annotations=asset.annotations,
                image_id=image_id,
                starting_annotation_id=annotation_id
            )

            if not metadata or "images" not in metadata or "annotations" not in metadata:
                print(f"⚠️ Skipping asset {asset.uuid} due to unexpected metadata format")
                continue

            combined_coco["images"].extend(metadata["images"])

            for ann in metadata["annotations"]:
                ann["id"] = annotation_id
                annotation_id += 1
                combined_coco["annotations"].append(ann)

            for cat in metadata.get("categories", []):
                if cat["name"] not in category_name_to_id:
                    cat_id = len(category_name_to_id) + 1
                    category_name_to_id[cat["name"]] = cat_id
                    combined_coco["categories"].append({
                        "id": cat_id,
                        "name": cat["name"],
                        "supercategory": "none"
                    })

        # Normalize category IDs across annotations
        name_to_id = {cat["name"]: cat["id"] for cat in combined_coco["categories"]}
        for ann in combined_coco["annotations"]:
            cat_id = ann["category_id"]
            cat_name = next((c["name"] for c in combined_coco["categories"] if c["id"] == cat_id), None)
            if cat_name:
                ann["category_id"] = name_to_id[cat_name]

        annotations_path = annotations_dir / "annotations.json"
        with open(annotations_path, "w") as f:
            json.dump(combined_coco, f, indent=2)

        await endpoint.disconnect()

    print("✅ Dataset downloaded successfully.")
    dataset = export_fiftyone_format(
        asset_list=asset_list_val,
        cache_dir=cache_directory,
        datasetUUID=datasetUUID
    )
    print("✅ FiftyOne dataset exported.")
    return dataset


def export_fiftyone_format(asset_list, cache_dir, datasetUUID):
    name = f"eyepop_dataset_{datasetUUID}"
    if name in fo.list_datasets():
        print(f"Dataset {name} already exists. Deleting it...")
        fo.delete_dataset(name)

    dataset = Dataset(name=name)

    for asset in asset_list:
        image_path = os.path.join(cache_dir, "data", f"{asset.uuid}.jpg")
        if not os.path.exists(image_path):
            continue

        sample = fo.Sample(
            filepath=image_path,
            tags=[asset.partition] if hasattr(asset, "partition") else [],
            metadata=ImageMetadata(),  # ✅ use correct type
        )

        # ✅ Store custom metadata as separate fields
        sample["partition"] = getattr(asset, "partition", None)
        sample["uuid"] = asset.uuid
        
        detections = []
        predictions_300 = []
        predictions_3k = []
        
        if hasattr(asset, "annotations"):
            for annotation in asset.annotations:
                if not hasattr(annotation, "annotation") or not hasattr(annotation.annotation, "objects"):
                    continue
                for obj in annotation.annotation.objects:
                    if(obj.classLabel != "stenosis"):
                        continue

                    if obj.confidence is None:
                        if annotation.type != "ground_truth":
                            continue
                    elif obj.confidence < 0.6:
                        continue

                    source = annotation.source if annotation.source is not None else ""
                    source_label= "(EyePop 300 Model)"
                    if "stenosis-ctktz" in source:
                        source_label= "(EyePop 3k Model)"

                    if annotation.type == "prediction" or annotation.type == "auto":
                        if(source_label == "(EyePop 300 Model)"):
                            predictions_300.append(
                                Detection(
                                    label=obj.classLabel  +" - "+ (source_label),
                                    bounding_box=[obj.x, obj.y, obj.width, obj.height],
                                    confidence=obj.confidence if hasattr(obj, "confidence") else 0
                                )
                            )
                        elif(source_label == "(EyePop 3k Model)"):
                            predictions_3k.append(
                                Detection(
                                    label=obj.classLabel  +" - "+ (source_label),
                                    bounding_box=[obj.x, obj.y, obj.width, obj.height],
                                    confidence=obj.confidence if hasattr(obj, "confidence") else 0
                                )
                            )
                    elif annotation.type == "ground_truth":
                        #if(annotation.source is None):
                        detections.append(
                            Detection(
                                label=obj.classLabel + " - " + ("ground_truth"),
                                bounding_box=[obj.x, obj.y, obj.width, obj.height],
                                confidence=1
                            )
                        )

        if detections:
            sample["ground_truth"] = Detections(detections=detections)
            
        if predictions_300:
            sample["predictions_300"] = Detections(detections=predictions_300)
        
        if predictions_3k:
            sample["predictions_3k"] = Detections(detections=predictions_3k)
        

        dataset.add_sample(sample)

    export_path = os.path.join(cache_dir, "fiftyone_dataset")
    dataset.export(
        export_dir=export_path,
        dataset_type=fo.types.FiftyOneDataset,
        label_field="ground_truth"
    )

    print(f"✅ FiftyOne dataset exported to {export_path}")
    return dataset

In [2]:
async def getEyePopModel(accountUUID, modelUUID):
    print("Fetching model from EyePop.ai...")
    print("Account UUID:", accountUUID)
    print("Model UUID:", modelUUID)
    print("Using API key:", apikey)

    async with EyePopSdk.dataEndpoint(
        eyepop_url="https://web-api.staging.eyepop.xyz/",
        secret_key=apikey,
        account_id=accountUUID,
        is_async=True,
        disable_ws=False,
    ) as endpoint:
        model = await endpoint.get_model(model_uuid=modelUUID)
        await endpoint.disconnect()

    if not model:
        raise ValueError(f"Model {modelUUID} not found for account {accountUUID}")

    return model

# model1 = await getEyePopModel(accountUUID, fullModelUUID)
# print(model1)

# model2 = await getEyePopModel(accountUUID, sub300ModelUUID)
# print(model2)

In [ ]:

async def run():
    print("Cache directory:", cache_directory)
    dataset = await downloadDatasetToCache(accountUUID, datasetUUID)
    await downloadDatasetToCache(accountUUID, sub300datasetUUID)

    for sample in dataset.take(10):
        print(sample.filepath, sample.ground_truth.detections)
       
    print("Loading dataset into FiftyOne...")    
    session = fo.launch_app(dataset, browser=True)
    
await run()

Cache directory: ./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/0688195453827ae280000526e30dc7a9
Account UUID: 034cb8e37f5444e98a78f1be65fd0bff
Dataset UUID: 068818885c547b1e80002649e95254ce
Cache directory: ./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/068818885c547b1e80002649e95254ce
Using API key: AAGcsWj8N2PlKQl9c9ydz3QFZ0FBQUFBQm1mZDB5eDUwalNlYi12NWotd3hsVGJiMW1sVXF1dE9aOU9oSGVBOWtBQXoxZmNjUE5Nb1YzY3RROUdzbVUwUkZtcDhZcG5vSWROTzR1TU8ybGhZckx6RTgzYVZwMjZEREZjalZubnpYaUNMWVdBODg9
Found 299 assets in the dataset.
Asset list saved to .cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/068818885c547b1e80002649e95254ce/eyepop_asset_list.json
Asset list VAL saved to .cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/068818885c547b1e80002649e95254ce/eyepop_asset_list_val.json


Starting downloads:   0%|          | 0/299 [00:00<?, ?it/s]

✅ Dataset downloaded successfully.
Dataset eyepop_dataset_068818885c547b1e80002649e95254ce already exists. Deleting it...
Directory './.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/068818885c547b1e80002649e95254ce/fiftyone_dataset' already exists; export will be merged with existing files
Exporting samples...
 100% |████████████████████| 121/121 [80.0ms elapsed, 0s remaining, 1.5K docs/s] 
✅ FiftyOne dataset exported to ./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/068818885c547b1e80002649e95254ce/fiftyone_dataset
✅ FiftyOne dataset exported.
Account UUID: 034cb8e37f5444e98a78f1be65fd0bff
Dataset UUID: 0688195453827ae280000526e30dc7a9
Cache directory: ./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/0688195453827ae280000526e30dc7a9
Using API key: AAGcsWj8N2PlKQl9c9ydz3QFZ0FBQUFBQm1mZDB5eDUwalNlYi12NWotd3hsVGJiMW1sVXF1dE9aOU9oSGVBOWtBQXoxZmNjUE5Nb1YzY3RROUdzbVUwUkZtcDhZcG5vSWROTzR1TU8ybGhZckx6RTgzYVZwMjZEREZjalZubnpYaUNMWVdBODg9
Found 3000 assets in the dataset.
Asset list saved 

Starting downloads:   0%|          | 0/3000 [00:00<?, ?it/s]

✅ Dataset downloaded successfully.
Dataset eyepop_dataset_0688195453827ae280000526e30dc7a9 already exists. Deleting it...
Directory './.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/0688195453827ae280000526e30dc7a9/fiftyone_dataset' already exists; export will be merged with existing files
Exporting samples...
 100% |████████████████████| 121/121 [68.2ms elapsed, 0s remaining, 1.8K docs/s] 
✅ FiftyOne dataset exported to ./.cache/voxel51/034cb8e37f5444e98a78f1be65fd0bff/0688195453827ae280000526e30dc7a9/fiftyone_dataset
✅ FiftyOne dataset exported.
Loading dataset into FiftyOne...
Connected to FiftyOne on port 5151 at localhost.
If you are not connecting to a remote session, you may need to start a new session and specify a port


In [4]:
import webbrowser

webbrowser.open("http://localhost:5151/")

True

In [5]:
import os
from pathlib import Path
import json

def extract_val_assets():
    dataset_path = Path(os.path.expanduser(cache_directory)) / "eyepop_asset_list_val.json"
    if not dataset_path.exists():
        print(f"Dataset file {dataset_path} does not exist.")
        return []

    with open(dataset_path, "r") as f:
        data = json.load(f)  # Load as full array
    
    assets = [Asset.model_validate(obj) for obj in data]  # or pydantic.parse_obj_as(List[Asset], data)
    return assets

def extract_coco_val_annotations(val_assets):
    annotations_path = Path(os.path.expanduser(cache_directory)) / "annotations" / "annotations.json"
    if not annotations_path.exists():
        print(f"Annotations file {annotations_path} does not exist.")
        return None

    with open(annotations_path, "r") as f:
        coco_data = json.load(f)

    # Filter annotations for the validation assets
    asset_uuids = {asset.uuid for asset in val_assets}
    coco_data["images"] = [img for img in coco_data["images"] if img["file_name"].replace(".jpg", "") in asset_uuids]

    #save filtered annotations
    filtered_annotations_path = Path(os.path.expanduser(cache_directory)) / "annotations" / "annotations_val.json"
    with open(filtered_annotations_path, "w") as f:
        json.dump(coco_data, f, indent=2)
    print(f"Filtered annotations saved to {filtered_annotations_path}")
    
    return coco_data

async def run():
    assets = extract_val_assets()
    print(f"Extracted {len(assets)} assets from the dataset.")
    #list all asset UUIDs
    # for asset in assets:
    #     print(f"Asset UUID: {asset.uuid}, Partition: {asset.partition}, Annotations: {len(asset.annotations)}")
    
    coco_annotations = extract_coco_val_annotations(assets)

    print("Loading dataset into FiftyOne...")    
    dataset = fo.Dataset.from_dir(
        dataset_dir=os.path.expanduser(cache_directory),
        dataset_type=fo.types.COCODetectionDataset,
        labels_path="annotations/annotations_val.json",
    )
    session = fo.launch_app(dataset, browser=True)

    # print the first 10 samples in the val set
    for sample in dataset.take(10):
        print(sample.filepath, sample.ground_truth.detections)
# await run()

Actions:
- Ensure github is setup with notebooks and data (dataset full, dataset 300, combine into fiftyone view)
- All thumbdrives loaded with github image
- Script to copy in model into person's account
- Test images for preview
- load 3k model onto snapdragon device with 10 images (same test images)
- 

# Step Setup
- Setup notebook 
- Setup EyePop account
- Copy API Key
- Copy in 3k model into everyone's account

# Notebook 01 
- Setup dataset and notebooks locally with thumbdrive or github
- load groundtruth of data into fiftyone
- Explore main values of FiftyOne: dataset exploration, visualization of predictions and model evaluation

# Notebook 02
Load dataset (300) into EyePop.ai training system and open to correct spot

# Step Break
Social & Eating time

# EP Product
- Explore EyePop.ai training system
- Explore Deploy and preview on EyePop.ai for model copied in
- AI Hub demo for deploying locally to Snapdragon devices

# Notebook 03 
- Re- Open fiftyone and Explore differences between 300 model and 3k model (and ground truth)
- More data for production 200k...


